In [ ]:
%load_ext autoreload
%autoreload 2

%reload_ext autoreload

In [6]:
!git status

On branch development
Your branch is up to date with 'origin/development'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	fonts

nothing added to commit but untracked files present (use "git add" to track)


In [ ]:
!git pull

In [4]:
!git commit -m "write function for hd panels" Screen_new.ipynb

On branch development
Your branch is ahead of 'origin/development' by 1 commit.
  (use "git push" to publish your local commits)

Untracked files:
	fonts

nothing added to commit but untracked files present


In [5]:
!git push

Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 4 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.76 KiB | 258.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To github.com:txoof/epdlib.git
   2bfe1fb..245136f  development -> development


In [ ]:
import logging
from PIL import Image, ImageDraw


In [ ]:
# import pkgutil

# import importlib
# import inspect

# import waveshare_epd
# from importlib import import_module
# non_hd = []
# for i in pkgutil.iter_modules(waveshare_epd.__path__):
#     non_hd.append(i.name)


# for i in non_hd:
    
#     if i.startswith('epd') and not i.startswith('epdconfig'):
#         print(i)
#         myepd = import_module(f'waveshare_epd.{i}')
#         try:
#             print('display: ',inspect.getfullargspec(myepd.EPD.display).args)
#         except AttributeError:
#             try:
#                 print('DISPLAY: ', inspect.getfullargspec(myepd.EPD.Display).args)
#             except AttributeError:
#                 print('display: unknown')
#         try:
# #             print('clear: ', inspect.getfullargspec(myepd.EPD.clear).args)
# #         except AttributeError:
#             print('CLEAR: ', inspect.getfullargspec(myepd.EPD.Clear).args)
#         except AttributeError:
#             print('clear: unknown')
            
#         print('\n')

In [ ]:
def strict_enforce(*types):
    """strictly enforce type compliance within classes
    
    Usage:
    @strict_enforce(type1, type2, (type3, type4))
    def foo(val1, val2, val4):
        ...
    """
    def decorator(f):
        def new_f(self, *args, **kwds):
            #we need to convert args into something mutable   
            newargs = []        
            for (a, t) in zip(args, types):
                if not isinstance(a, t):
                    raise TypeError(f'"{a}" is not type {t}')
#                 newargs.append( t(a)) #feel free to have more elaborated convertion
            return f(self, *args, **kwds)
        return new_f
    return decorator

In [ ]:
class ScreenError(Exception):
    pass

In [ ]:
class Screen():
    def __init__(self, epd='None', rotation=0, mode='1', vcom=0.0):
        self.vcom = vcom
        self.constants = None
        self.mode = mode
        self.image = None
        self.hd = False
        self.resolution = []
        self.HD = False
        self.epd = epd
        self.rotation = rotation
        
        
    @property
    def vcom(self):
        return self._vcom

    @vcom.setter
    @strict_enforce(float)
    def vcom(self, vcom):
        if vcom==0:
            self._vcom = None
        elif vcom > 0:
            raise ValueError(f'vcom must be a negative float value: {vcom}')
        else:
            self._vcom = vcom

    @property
    def epd(self):
        return self._epd

    @epd.setter
    @strict_enforce(str)
    def epd(self, epd):
        myepd = None
        if epd=='HD':
            if not self.vcom:
                raise ScreenError('no vcom value is set (see the cable on your display for a vcom value)')
            self.HD = True
            myepd = self._epd_hd(epd)
            
        elif epd == 'None':
            myepd = None
        else:
            myepd = self._epd_non_hd(epd)

        if myepd:
            # set the resolution 
            self._epd = myepd['epd']
            resolution = myepd['resolution']
            resolution.sort(reverse=True)
            self.resolution = resolution
            self.clear_args = myepd['clear_args']
            self.constants = myepd['constants']
            
            # set a blank image as default
            self.image = Image.new('L', self.resolution, 255)
            if self.HD:
                self.buffer_no_image = []
            else:
                self.buffer_no_image = self.epd.getbuffer(self.blank_image())
        else:
            logging.warning('no valid epd is currently configured')
            
    def initEPD(self):
        '''init the EPD for writing'''
        if not self.epd:
            raise UnboundLocalError('no epd object has been assigned')
            
        if self.HD:
            self._epd.epd.run()
        else:
            try:
                self.epd.init()
            except FileNotFoundError as e:
                raise ScreenError('failed to open SPI bus - is spi enabled in raspi-config?')
#                 logging.error(f'failed to init epd: {e}: error: {type(e)}')

        logging.info(f'epd initialized')
            
        return True            

    def clearEPD(self):
        if self.HD:
            self._epd.epd.run()
            self._epd.clear()
        else:
            try:
                self.initEPD()
                self.epd.Clear(**self.clear_args)
            # FIXME -- more explicit output here on failure
            except Exception as e:
                logging.error(f'failed to clear epd: {e}')
                return False
        return True
        
    
    def blank_image(self):
        '''generate PIL image that is entirely blank'''
        return Image.new(self.mode, self.resolution, 255)
            
            

    
    def writeEPD(self, image=None, sleep=True):
        '''write an image to the screen after clearing previous
        
            Non-hd screens should be put to sleep after writing to prevent
            damage to the panel.
        
        Args:
            image(PIL image): image to display
            sleep(bool) Put display to sleep after updating
            
            '''
        if not self.epd:
            raise UnboundLocalError('no epd has been assigned')
            
        if self.HD:
            self._writeEPD_hd(self, image, sleep)
        else:
            self._writeEPD_non_hd(self, image, sleep)
            
        
    
    def _writeEPD_hd(self, image):
        '''redraw entire screen, no partial update'''
        # not entirely sure what this is for
        self.epd.frame_buf.paste(0xFF, box=(0, 0, self.resolution[0], self.resolution[1]))
        
        self.epd.frame_buf.paste(image, [0,0])
        # see https://www.waveshare.net/w/upload/c/c4/E-paper-mode-declaration.pdf for display modes

        self.epd.frame_buf.paste(image, [0, 0])

        self.initEPD()
        self.epd.draw_full(s.constants.DisplayModes.GC16)
        return True
        
    def _writeEPD_non_hd(self):
        pass

    
    def _epd_hd(self, epd):
        from IT8951.display import AutoEPDDisplay
        from IT8951 import constants as constants_HD
        myepd = AutoEPDDisplay(vcom=self.vcom)
        resolution = list(myepd.display_dims)
        clear_args = {}
        one_bit_display = False
        
        return {'epd': myepd, 
                'resolution': resolution, 
                'clear_args': clear_args, 
                'one_bit_display': one_bit_display,
                'constants': constants_HD}    
                    
    def _epd_non_hd(self, epd):
        import waveshare_epd
        import pkgutil
        import inspect
        from importlib import import_module
        non_hd = []
        for i in pkgutil.iter_modules(waveshare_epd.__path__):
            non_hd.append(i.name)

        if epd in non_hd:
            myepd = import_module(f'waveshare_epd.{epd}')
            resolution = [myepd.EPD_HEIGHT,myepd.EPD_WIDTH]
            
            # set kwargs for screens that expect color or mode arguments to the clear function
            try:
                clear_args_spec = inspect.getfullargspec(myepd.EPD.Clear)
            except AttributeError:
                raise ScreenError(f'"{epd}" has an unsupported `EPD.Clear()` function')
            clear_args = {}
            if 'color' in clear_args_spec:
                clear_args['color'] = 0xFF
                
            try:
                display_args_spec = inspect.getfullargspec(myepd.EPD.display)
            except AttributeError:
                raise ScreenError(f'"{epd}" has an unsupported `EPD.display()` function')
            if len(display_args_spec) > 2:
                one_bit_display = True
            else:
                one_bit_display = False
        else:
            raise ScreenError(f'invalid waveshare module: {epd}')
            
        return {'epd': myepd.EPD(), 
                'resolution': resolution, 
                'clear_args': clear_args,
                'one_bit_display': one_bit_display,
                'constants': None}

In [ ]:
# # add this to the hd write function
# s.epd.frame_buf.paste(0xFF, box=(0, 0, s.epd.width, s.epd.height))

# i = mylayout_hd.concat()
# s.epd.frame_buf.paste(i, [0, 0])

# s.initEPD()
# s.epd.draw_full(s.constants.DisplayModes.GC16)
# # s.epd.clear()

In [ ]:
# s.epd.clear()
# s.epd.epd.sleep()

In [ ]:
import Layout

In [ ]:
l = {
    'text_a': {
        'image': None,
        'padding': 10, 
        'width': 1,
        'height': 1/4,
        'abs_coordinates': (0, 0),
        'mode': 'L',
        'font': './fonts/Open_Sans/OpenSans-ExtraBold.ttf',
        'max_lines': 1,
        'font_size': None,
        'fill': 128},
    
    'text_b': {
        'image': None,
        'padding': 10,
        'inverse': True,
        'width': 1,
        'height': 1/4,
        'abs_coordinates': (0, None),
        'relative': ['text_b', 'text_a'],
        'mode': 'L',
        'font': './fonts/Open_Sans/OpenSans-ExtraBold.ttf',
        'max_lines': 3,
        'font_size': None},
    
    'image_a': {
        'image': True,
        'width': 1/2,
        'height': 1/2,
        'mode': 'L',
        'abs_coordinates': (0, None),
        'relative': ['image_a', 'text_b'],
        'scale_x': 1,
        'hcenter': True,
        'inverse': True},
    
    'image_b': {
        'image': True,
        'width': 1/2,
        'height': 1/2,
        'mode': 'L',
        'abs_coordinates': (None, None),
        'relative': ['image_a', 'text_b'],
        'bkground': 0,
        'vcenter': True},
        
}

u1 = {'text_a': 'The quick brown fox jumps over the lazy dog.',
     'text_b': 'Pack my box with five dozen liquor jugs. Jackdaws love my big sphinx of quartz.',
     'image_a': '../PIA03519_small.jpg',
     'image_b': '../hubble.jpg'}

In [ ]:
epd2in7 = Screen(epd='epd2in7')
# epd2in7.clearEPD()

In [ ]:
mylayout_non = Layout.Layout(resolution=epd2in7.resolution, layout=l)

In [ ]:
epd2in7.initEPD()
epd2in7.clearEPD()

In [ ]:
s = Screen(epd='HD', vcom=-1.93, mode='L')

In [ ]:
mylayout_hd = Layout.Layout(resolution=s.resolution, layout=l)
mylayout_hd.update_contents(u1)
mylayout_hd.concat()

In [ ]:
mylayout_hd.update_contents(u1)
mylayout_hd.concat()
s._writeEPD_hd(mylayout_hd.image)

In [ ]:
s.initEPD()
s.clearEPD()

In [ ]:
logger = logging.getLogger(__name__)
logger.root.setLevel('DEBUG')